In [1]:
import torch
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from torchvision import datasets, transforms
from tqdm import tqdm

/home2/sanika/miniconda3/envs/smai/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import wandb

In [3]:
# download the cifar10 dataset
trainset = datasets.CIFAR10(root='/scratch/cifar10', train=True, download=True, transform=ToTensor())

100%|██████████| 170498071/170498071 [00:33<00:00, 5043910.66it/s]


Extracting /scratch/cifar10/cifar-10-python.tar.gz to /scratch/cifar10


In [4]:
testset = datasets.CIFAR10(root='/scratch/cifar10', train=False, download=False, transform=ToTensor())

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
class VGG16(nn.Module):
    def __init__(self, num_classes, batchnorm):
        super(VGG16, self).__init__()
        self.layer1 = self._make_layer(3, 64, batchnorm)
        self.layer2 = self._make_layer(64, 64, batchnorm, maxpool=True)
        self.layer3 = self._make_layer(64, 128, batchnorm)
        self.layer4 = self._make_layer(128, 128, batchnorm, maxpool=True)
        self.layer5 = self._make_layer(128, 256, batchnorm)
        self.layer6 = self._make_layer(256, 256, batchnorm)
        self.layer7 = self._make_layer(256, 256, batchnorm, maxpool=True)
        self.layer8 = self._make_layer(256, 512, batchnorm)
        self.layer9 = self._make_layer(512, 512, batchnorm)
        self.layer10 = self._make_layer(512, 512, batchnorm, maxpool=True)
        self.layer11 = self._make_layer(512, 512, batchnorm)
        self.layer12 = self._make_layer(512, 512, batchnorm)
        self.layer13 = self._make_layer(512, 512, batchnorm, maxpool=True)
        self.layer14 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 4096),
            nn.ReLU(),
        )
        self.layer15 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
        )
        self.layer16 = nn.Sequential(
            nn.Linear(4096, num_classes),
        )

    def _make_layer(self, in_channels, out_channels, batchnorm, maxpool=False):
        layers = [
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=1),
        ]
        
        if batchnorm:
            layers.append(nn.BatchNorm2d(out_channels))
            
        layers.append(nn.ReLU())
        
        if maxpool:
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
            
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.view(out.size(0), -1)
        out = self.layer14(out)
        out = self.layer15(out)
        out = self.layer16(out)
        return out
   
    def fit(self, loaders, epochs, device, learning_rate):
        self.to(device)
        criterion = nn.CrossEntropyLoss().to(device)
        # adamoptimizer
        optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        for epoch in range(epochs):
            self.train().to(device)

            total_loss = 0
            correct = 0
            total = 0

            for i, (images, labels) in enumerate(loaders['train']):
                self.train().to(device)
                optimizer.zero_grad()

                images = images.to(device)
                labels = labels.to(device)
                
                outputs = self(images)
                loss = criterion(outputs, labels)
                
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                # print accuracy and loss for each batch
                # save the loss and accuracy into a file
                if(i % 30 == 0):
                    # open file for training accuracy
                    wandb.log({"Epoch": epoch + 1, "Loss": total_loss / (i + 1), "Train_Accuracy": 100 *correct / total})
                    # ftrain.write(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {total_loss / (i + 1)}, Accuracy: {100 * correct / total}\n')

                # set to eval mode, find accuracy on trainset
                if(i % 30 == 0):
                    self.eval().to(device)
                    test_total = 0
                    test_correct = 0
                    for j, (imagestest, labelstest) in enumerate(loaders['test']):
                        imagestest = imagestest.to(device)
                        labelstest = labelstest.to(device)
                        outputstest = self(imagestest)
                        _, predictedtest = torch.max(outputstest.data, 1)
                        test_total += labelstest.size(0)
                        test_correct += (predictedtest == labelstest).sum().item()
                    # print accuracy and loss for each batch
                    # save the loss and accuracy into a file
                    wandb.log({"Epoch": epoch + 1, "Loss": total_loss / (i + 1), "TestAccuracy": 100 * test_correct / test_total})
                    # ftest.write(f'Epoch: {epoch + 1}, Batch: {i + 1}, Accuracy: {100 * test_correct / test_total}\n')
                    
            total_loss = total_loss / len(loaders['train'])
            accuracy = 100 * correct / total
            print('Epoch: {}, Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch + 1, total_loss, accuracy))
